In [ ]:
def all_models(start, end, patient_id):
    models = []
    for start_ in np.arange(start, end, 50):
        for end_ in np.arange(start_ + 100, end + 1, 100):
            for reg_method in [2]:
                models.append(find_T(start_, end_, patient_id, reg_method))
    return models

Period = namedtuple('Period', ['start', 'end'])
def all_periods(start, end):
    periods = []
    for start_ in np.arange(start, end, 100):
        end_ = start_ + 100
        period = Period(start_, end_)
        periods.append(period)
    return periods

def models_periods_matrix(start, end, patient_id):
    
    if os.path.exists('pickles/mat'+str(patient_id)+'-'+str(start)+'-'+str(end)) == True:
        mat = pickle.load(open('pickles/mat'+str(patient_id)+'-'+str(start)+'-'+str(end), 'rb'))
        models = pickle.load(open('pickles/models'+str(patient_id)+'-'+str(start)+'-'+str(end), 'rb'))
        periods = pickle.load(open('pickles/periods'+str(patient_id)+'-'+str(start)+'-'+str(end), 'rb'))
        return mat, models, periods
    
    models = all_models(start, end, patient_id)
    periods = all_periods(start, end)

    mat = np.empty((len(periods),len(models)), dtype=object)
    print(mat.shape)

    for i in range(len(models)):
        for j in range(len(periods)):
            prediction = test_T(models[i], periods[j].start, periods[j].end, patient_id)
            mat[j][i] = (prediction.data_test, prediction.data_pred)
            
    with open('pickles/mat'+str(patient_id)+'-'+str(start)+'-'+str(end), 'wb') as mat_file:
        pickle.dump(mat, mat_file)
    with open('pickles/models'+str(patient_id)+"-"+str(start)+"-"+str(end), 'wb') as models_file:
        pickle.dump(models, models_file)
    with open('pickles/periods'+str(patient_id)+"-"+str(start)+"-"+str(end), 'wb') as periods_file:
        pickle.dump(periods, periods_file)
        
    return mat, models, periods

def translate_matrix(mat, models, periods, ac_limit, mean_limit):
    clean_mat = np.empty((len(periods),len(models)), dtype=object)
    for i in range(len(models)):
        for j in range(len(periods)):
            (data_test, data_pred) = mat[j][i]
            error = data_test - data_pred
            
            left_rank_mean = right_rank_mean = rank_mean = extreme_mean = 0
            rank_ac = 0 # autocorelation
            
            for signal in [0,1,2]:
                if np.abs(np.mean(error[signal])) <= mean_limit:
                    rank_mean = rank_mean + 1
                if np.abs(np.mean(error[signal])) <= 0.4:
                    extreme_mean = extreme_mean + 1
                if np.abs(np.mean(error[signal][0:(int)(len(error[signal])/2)])) <= 0.7:
                    left_rank_mean = left_rank_mean + 1
                if np.abs(np.mean(error[signal][(int)(len(error[signal])/2):])) <= 0.7:
                    right_rank_mean = right_rank_mean + 1
                    
                h, pV, Q, cV = lbqtest(error[signal], range(1, min(20, len(error[signal]))), alpha=ac_limit)
                if not any(h):      
                    rank_ac = rank_ac + 1

            if (rank_ac == 3 and rank_mean == 3) or extreme_mean == 3:# and left_rank_mean == 3 and right_rank_mean == 3:
                clean_mat[j][i] = True
            else:
                clean_mat[j][i] = False
    return clean_mat


def clean_matrix(mat, models, periods):
    if len(periods) == 1:
        return mat
    clean_mat = np.empty((len(periods),len(models)), dtype=object)
    for i in range(len(models)):
        for j in range(len(periods)):
            clean_mat[j][i] = mat[j][i]
            if mat[j][i] == True:
                if (j == 0 and mat[1][i] == False) or (j == (len(periods) - 1) and mat[len(periods) - 2][i] == False) or ((j!=0) and (j!=(len(periods) - 1)) and (mat[j-1][i] == False) and (mat[j+1][i]==False)):
                    clean_mat[j][i] = False
    return clean_mat


def popular_model(mat, models, periods):
    Period = namedtuple('Period', ['start', 'end'])
    models_popularity = np.zeros(len(models))
    model_periods = []
    for i in range(len(models)):
        pre_fit = False
        for j in range(len(periods)):
            if pre_fit == True and mat[j][i] == True:
                models_popularity[i] = models_popularity[i] + 1
            if mat[j][i] == True:
                models_popularity[i] = models_popularity[i] + 1
                pre_fit = True
            else:
                pre_fit = False

    print("models_popularity", models_popularity)
    popular_index = np.argmax(models_popularity)
    print("popular_index", popular_index)
    to_delete = []
    for i in range(len(periods)):
        if mat[i][popular_index] == True:
            if len(model_periods) == 0 or model_periods[-1].end != periods[i].start:
                model_periods.append(periods[i])
            else:
                last_period_start = model_periods[-1].start
                model_periods.pop(-1)
                model_periods.append(Period(last_period_start, periods[i].end))
            to_delete.append(i)
    for i in reversed(to_delete):
        mat = np.delete(mat, i, axis=0)
        periods.pop(i)
    return mat, periods, models[popular_index], model_periods



Model_Periods = namedtuple('Model_Periods', ['model', 'model_periods'])
def split_to_models(start, end, patient_id):
    
    for ac_limit in [0.2, 0.15, 0.1, 0.05, 0.03, 0.01]:
        for mean_limit in [0.45, 0.5, 0.55, 0.6]:
            print("ac_limit, mean_limit = ", ac_limit, mean_limit)
            mat, models, periods = models_periods_matrix(start, end, patient_id)
            mat = translate_matrix(mat, models, periods, ac_limit, mean_limit)
            mat = clean_matrix(mat, models, periods)

            results = []
            for i in range(3):
                mat, periods, popular_model_, model_periods = popular_model(mat, models, periods)
                if len(model_periods) == 0:
                    break;
                else:
                    results.append(Model_Periods(popular_model_, model_periods))

            if len(periods)*100 <= 0.2*(end-start):
                for result in results:
                    for period in result.model_periods:
                        print_prediction_results(test_T(result.model, period.start, period.end, patient_id))
                return;